<a href="https://colab.research.google.com/github/mmelicher91/ITSD_rozpis/blob/main/itsd_svatky_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import calendar
from tabulate import tabulate

###################### Svátky 2023: [měsíc, den] ###############################
svatky=[[1,1],[5,1],[5,8],[7,5],[7,6],[9,28], 
        [10,28],[11,17],[12,24],[12,25],[12,26],
        [4,7],[4,10]] # <-- velikonoce 7.4.2023 a 10.4.2023

######################### Nastavení osob #######################################
barveni = {"LAN":"#ddd9c4","ŠTO":"#92cddc","PAL":"#c4d79b","ITIN":"#e26b0a"}

osoby = [["ŠTO","#92cddc",0],
         ["LAN","#ddd9c4",1],
         ["PAL","#c4d79b",2],
         ["ITIN","#e26b0a",3]]     

########################### Statická data ######################################
dny_v_tydnu= ("Po ","Út ","St ","Čt ","Pá ","So ","Ne ")
nazvy_mesicu =("Leden","Únor","Březen", "Duben", "Květen", "Červen", "Červenec", "Srpen", "Září", "Říjen", "Listopad", "Prosinec")
nadpis = ("DEN","RANO","ODPO"," ","POH","   ")


################################################################################
################################# Clovek #######################################
################################################################################

class Clovek():
  # Generuje předchůdce a následovníka, rozlišuje lichý vs sudý počet osob
  def __init__(self,p):
    self.index=p
    self.znacka=osoby[p][0]
    self.barva=osoby[p][1]
    # Podmínka: LICHÝ vs SUDÝ počet osob
    if len(osoby)%2==0:
      if p%2==0: 
        # Sudý řádky posunu o +1 hodnotu vpřed
        self.predchudce=osoby[p-len(osoby)+1][0]
        # Podmínka: NASLEDUJICI: poslední sudý řádek se odkáže na začátek osoby[index=0]
        if len(osoby)-2==p:  
          self.nasledujici=osoby[len(osoby)-p-2][0]
        else: 
          # Není poslední sudý řádek .. skáču o +3 indexy
          self.nasledujici=osoby[p+3-len(osoby)][0]      
      else: 
        # Lichý řádky: Posunu se o -1 hodnotu zpět
        self.predchudce=osoby[p-len(osoby)-1][0]
        # Podmínka: NASLEDUJICI: poslední lichý řádek se odkáže na první lichý záznam (tj. začátek osoby[index=1])
        if len(osoby)-1==p: 
          self.nasledujici=osoby[len(osoby)-p][0]
        else:
          self.nasledujici=osoby[p+1-len(osoby)][0]
    else: 
      # Lichý počet osob: řazení je automatické, netřeba ošetřovat
      self.predchudce=osoby[p-1][0]
      self.nasledujici=osoby[p-len(osoby)+1][0]

################################################################################
################################# Rozpis #######################################
################################################################################

class Rozpis:
  def __init__(self,p):
    self.index=p
    self.rok=rok
    self.mesic=p+1
    self.nazev=nazvy_mesicu[p]
    kal = calendar.monthrange(self.rok,self.mesic)
    self.prvni_den = kal[0]
    self.jmeno_dne = dny_v_tydnu[self.prvni_den]
    self.delka_dnu = kal[1]
    
    pocet=[]
    for _ in range(self.delka_dnu):
      pocet.append(_+1)
    self.pocet=pocet

    self.matice= np.zeros((self.delka_dnu,6),dtype=object)
    
    for k in range(len(svatky)):
      if svatky[k][0] == self.mesic:
        for jk in range(self.delka_dnu):
          if svatky[k][1] == self.pocet[jk]:
            self.matice[jk,1]="X"
            self.matice[jk,2]="X"
 
################################################################################ 

def rozpis_dnu(den,poc,posledni,poho):
  poradi=1 # rozhoduje o použití PREDCHUDCE (poradi=-1) x NASLEDOVNIK (poradi=1)
  for k in range(len(osoby)):
    if poho == osoby[k][0]:
    #Pokud poslední POHO v měsíci je neděle, tak posune o index dál
      if (k == len(osoby)-1) and (den==0):
        idx=0
        break
      elif (k == len(osoby)-1):
        idx=0
      else:
        idx=k+1

  for i in range(poc):
    for j in range(2):
      mesic[x].matice[i,0]=i+1 #dny_v_tydnu[den] <-- vypsání dnů
      if (5 <= den <= 6):
        mesic[x].matice[i,j+1]="" # So/Ne Ráno/Odpo
        mesic[x].matice[i,5]="" # prázdný sloupec
      else:
        posledni=predej(posledni,poradi)
        if mesic[x].matice[i,j+1]=="X":
          mesic[x].matice[i,j+1]=""
          mesic[x].matice[i,5]="sv"          
        else:
          mesic[x].matice[i,j+1]=posledni
          mesic[x].matice[i,5]=""
          # poradi se meni pouze u sudych osob
        if len(osoby)%2==0:
          poradi*=-1
    
    mesic[x].matice[i,4]=osoby[idx][0]
    mesic[x].matice[i,3]="" # prázdný sloupec

    if den == 6:
      den=0
      if idx == len(osoby)-1:
        idx=0
      else:
        idx+=1
    else:
      den+=1

def predej(posledni,poradi):
  for kl in range(len(osoby)):
    if posledni==clovek[kl].znacka:
      if poradi==-1:
        posledni=clovek[kl].predchudce
      else:
        posledni=clovek[kl].nasledujici
      break  
  return posledni

############################# Barveni PANDAS ###################################
def obarvi(col, pattern_map, default=''):
  return np.select(
      [col.str.contains(k, na=False) for k in pattern_map.keys()],
      [f'background-color: {v}' for v in pattern_map.values()],
      default=default, 
    ).astype(str)
################################################################################
obj = calendar.Calendar()
###################### Vyprázdněný Nulovaní při spuštění #######################
mesic=[]
clovek=[]
pocet=[]
################################################################################
rok=int(input("Zadej rok: "))
x=int(input("Zadej měsíc: "))-1
posledni=input("Zadej poslední odpolední službu: ").upper()
poho=input("Zadej poslední pohotovost: ").upper()


for p in range(12):
  mesic.append(Rozpis(p))
 
for p in range(len(osoby)):
  clvk=Clovek(p)
  clovek.append(clvk)

#for i in range(len(osoby)):
# print(vars(clovek[i]))

poc=mesic[x].delka_dnu
den=mesic[x].prvni_den
data = pd.DataFrame(mesic[x].matice, index=mesic[x].pocet, columns=nadpis)
rozpis_dnu(den, poc, posledni,poho)

kontrola= True
while kontrola == True:
  otazka=int(input("\nVyber akci: 0 - Grafický výstup | 1 - Export do excelu | 2 - Textový výstup | 9 - Ukončit program \n >> "))
  if otazka == 0:
    print(f"{mesic[x].nazev} {mesic[x].rok}")
    display(data.style.apply(obarvi,pattern_map=barveni,subset=["RANO","ODPO","POH"]))
    print("")
  elif otazka == 1:
    data.style.apply(obarvi, pattern_map=barveni, subset=["RANO","ODPO","POH"]).to_excel("report.xlsx",sheet_name=mesic[x].nazev, index=False)
  elif otazka == 2:
    print(tabulate(mesic[x].matice, tablefmt="github", showindex="none"))
  else:
    print("\nUkončuji program...")
    break

  




Zadej rok: 2023
Zadej měsíc: 5
Zadej poslední odpolední službu: lan
Zadej poslední pohotovost: što

Vyber akci: 0 - Grafický výstup | 1 - Export do excelu | 2 - Textový výstup | 9 - Ukončit program 
 >> 0
Květen 2023


,DEN,RANO,ODPO,,POH,
1,1,,,,LAN,sv
2,2,LAN,ŠTO,,LAN,
3,3,ITIN,PAL,,LAN,
4,4,ŠTO,LAN,,LAN,
5,5,PAL,ITIN,,LAN,
6,6,,,,LAN,
7,7,,,,LAN,
8,8,,,,PAL,sv
9,9,ITIN,PAL,,PAL,
10,10,ŠTO,LAN,,PAL,




Vyber akci: 0 - Grafický výstup | 1 - Export do excelu | 2 - Textový výstup | 9 - Ukončit program 
 >> 2
|----|------|------|--|------|----|
|  1 |      |      |  | LAN  | sv |
|  2 | LAN  | ŠTO  |  | LAN  |    |
|  3 | ITIN | PAL  |  | LAN  |    |
|  4 | ŠTO  | LAN  |  | LAN  |    |
|  5 | PAL  | ITIN |  | LAN  |    |
|  6 |      |      |  | LAN  |    |
|  7 |      |      |  | LAN  |    |
|  8 |      |      |  | PAL  | sv |
|  9 | ITIN | PAL  |  | PAL  |    |
| 10 | ŠTO  | LAN  |  | PAL  |    |
| 11 | PAL  | ITIN |  | PAL  |    |
| 12 | LAN  | ŠTO  |  | PAL  |    |
| 13 |      |      |  | PAL  |    |
| 14 |      |      |  | PAL  |    |
| 15 | ITIN | PAL  |  | ITIN |    |
| 16 | ŠTO  | LAN  |  | ITIN |    |
| 17 | PAL  | ITIN |  | ITIN |    |
| 18 | LAN  | ŠTO  |  | ITIN |    |
| 19 | ITIN | PAL  |  | ITIN |    |
| 20 |      |      |  | ITIN |    |
| 21 |      |      |  | ITIN |    |
| 22 | ŠTO  | LAN  |  | ŠTO  |    |
| 23 | PAL  | ITIN |  | ŠTO  |    |
| 24 | LAN  | ŠTO  |  | ŠTO  